# DMA測試
資料來源：(論壇1)[https://discuss.pynq.io/t/tutorial-pynq-dma-part-1-hardware-design/3133] (論壇2)[https://discuss.pynq.io/t/tutorial-pynq-dma-part-2-using-the-dma-from-pynq/3134] (學長提供的資料)[https://drive.google.com/drive/folders/1rx_Gwq8f9XHVFbKQO87_0Q4z6Ag4Frl1]

載入由Vivado產生的bit、hwh檔案
分別位於：
{ project_location }/{project_name}.runs/impl_1/design_1_wrapper.bit 與 
{ project_location }/{project_name}.gen/sources_1/bd/design_1/hw_handoff
/design_1.hwh
`注意：.hwh與.bit檔案名稱需相同`

In [1]:
from pynq import Overlay

ol = Overlay("zynq.bit")

In [15]:
ol.ip_dict

{'dma': {'type': 'xilinx.com:ip:axi_dma:7.1',
  'mem_id': 'S_AXI_LITE',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_LITE_ADDR_WIDTH': '10',
   'C_S_AXI_LITE_DATA_WIDTH': '32',
   'C_DLYTMR_RESOLUTION': '125',
   'C_PRMRY_IS_ACLK_ASYNC': '0',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM_CHANNELS': '1',
   'C_INCLUDE_SG': '0',
   'C_SG_INCLUDE_STSCNTRL_STRM': '0',
   'C_SG_USE_STSAPP_LENGTH': '0',
   'C_SG_LENGTH_WIDTH': '14',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_S_AXIS_S2MM_STS_TDATA_WIDTH': '32',
   'C_MICRO_DMA': '0',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_SF': '1',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXI_MM2S_ADDR_WIDTH': '32',
   'C_M_AXI_MM2S_DATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_INCLUDE_MM2S_DRE': '0',
   'C_INCLUDE_S2MM': '1',
   'C_INCLUDE_S2MM_SF': '1',
   'C_S2MM_BURST_SIZE': '16',
   'C_M_

In [16]:
ol?

In [17]:
ol.dma?

In [18]:
dma = ol.dma
dma_send = ol.dma.sendchannel
dma_recv = ol.dma.recvchannel

In [19]:
from pynq import allocate
import numpy as np

data_size = 1000
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)

In [20]:
for i in range(data_size):
    input_buffer[i] = i + 0xcafe0000

In [21]:
for i in range(10):
    print(hex(input_buffer[i]))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


In [22]:
dma_send.transfer(input_buffer)

In [23]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


接收DMA傳回的資料

In [24]:
dma_recv.transfer(output_buffer)

In [25]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


In [26]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

Arrays are equal: True
